# Projet maison n° 3

In [1]:
# imports
import pandas as pd
import os
import re
import numpy as np

In [2]:
DATA_DIR = "data"
dirname_path = f"{DATA_DIR}/names"

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [3]:
def extract_year(text):
    """ Extract year within text string """
    return re.findall(r'\d+', text)[0]

In [4]:
us_filename_path = dirname_path + "/yob{year}.txt"

years = [extract_year(filename) for filename in os.listdir(dirname_path) if ((os.path.isfile(os.path.join(dirname_path, filename))) and (os.path.splitext(filename)[-1] == '.txt'))]
print(f"{len(years)} files retrieved")

142 files retrieved


In [5]:
# names us
def df_names_us(filename_path=us_filename_path, years=years, verbose=False):
    """ Create df with US baby names """
    df = pd.DataFrame()
    
    # for year in sorted(years):
    for year in years:
        # Load text content to df
        filename = filename_path.format(year=year)
        if verbose:
            print("filename", filename)
        df_tmp = pd.read_csv(filename, sep=",", header=None)
        
        # Order columns and add 'year' column
        df_tmp.columns = ['name', 'gender', 'births']
        df_tmp['year'] = year
        
        # Concatenate both df
        df = pd.concat([df, df_tmp], ignore_index=True)
        
    # Reorder columns
    df = df[['year', 'name', 'gender', 'births']]
    
    # Sort columns 'year': -+, 'gender' -+, 'births' +- and 'name' -+
    df = df.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])
    
    # Set columns type to string and int
    # df = df.astype({
    #     # 'year': 'int',
    #     'name':'str',
    #     'gender': 'str',
    #     'births': 'int'
    # })
    df = df.reset_index(drop=True)

    return df

df_us = df_names_us()
df_us

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
2052776,2021,Zyeire,M,5
2052777,2021,Zyel,M,5
2052778,2021,Zyian,M,5
2052779,2021,Zylar,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [6]:
# names fr
def df_names_fr(dirname_path=dirname_path, verbose=False):
    """ Create df with FR baby names """
    
    # Load csv file
    fr_filename_path = dirname_path + "/nat2021.csv"
    df = pd.read_csv(fr_filename_path, sep=";")

    # Rename and reorder columns
    df = df.rename(columns={
        'sexe':'gender',
        'preusuel':'name',
        'annais':'year',
        'nombre':'births'
    })
    df = df[['year', 'name', 'gender', 'births']]

    # Set gender columns type to string and int
    df = df.astype({
        # 'year': 'int',
        'name':'str',
        'gender': 'str',
        'births': 'int'
    })

    # Map gender values (1->F, 2->M)
    df['gender'] = df['gender'].map({"1":"F", "2":"M"})

    # Remove rows with len(name)<2
    df = df[df['name'].str.len()>=2]

    # Apply title case to names
    df['name'] = df['name'].apply(lambda x: x.title())

    # Remove noisy rows
    df = df[df['year'].str.isnumeric().fillna(True)]
    df = df[~df['name'].str.contains('_')]

    # Sort columns 'year': -+, 'gender' -+, 'births' +- and 'name' -+
    df = df.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])

    # Reset index
    df = df.reset_index(drop=True)
    
    # Set columns type to string and int
    # df = df.astype({
    #     # 'year': 'int',
    #     'name':'str',
    #     'gender': 'str',
    #     'births': 'int'
    # })

    return df

df_fr = df_names_fr()
df_fr

,year,name,gender,births
0,1900,Jean,F,14097
1,1900,Louis,F,9052
2,1900,Pierre,F,7456
3,1900,Joseph,F,7259
4,1900,Henri,F,6921
...,...,...,...,...
648325,2021,Éoline,M,3
648326,2021,Éponine,M,3
648327,2021,Érica,M,3
648328,2021,Éthel,M,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [7]:
# taux de change
def df_taux_change(devises, verbose=False):
    """ Create df for exchange rate over time for specific currencies """
    
    # Load csv file
    webstat_export_filename_path = dirname_path + "/Webstat_Export.csv"
    if verbose:
        print(webstat_export_filename_path)
    df = pd.read_csv(webstat_export_filename_path, sep=";")

    # Process df
    df.index = df['Titre :']
    df = df.drop(columns=['Titre :'])
    df = df.T

    # Extract ISO numero
    df['devises'] = df['Unité :'].str.split('(').str[-1].str.split(')').str[0] #.str.extract('\((.*)\)')
    df = df.T
    df.columns = df.loc['devises'].values

    # Remove useless columns and rows with NaN values
    df = df.drop([
        'Code série :',
        'Unité :',
        'Magnitude :',
        "Méthode d'observation :",
        'Source :',
        'devises'
    ])
    df = df.replace('-', np.nan)
    df = df.loc[~df.isna().any(axis=1)]
    
    # Convert df columns from string to float
    df = df.apply(lambda x: x.str.replace(',', '.').astype(float), axis=1)

    # Convert index into datetime type
    df.index = pd.to_datetime(df.index, format="%d/%m/%Y")
    df.index.name = None

    # Sort index by ascending values
    df = df.sort_index(ascending=True)

    # Select columns
    df = df[devises]
    
    return df

df_exchange_rate = df_taux_change(devises=["CHF", "GBP", "USD"])
df_exchange_rate

,CHF,GBP,USD
2000-07-19,1.5473,0.61560,0.9216
2000-07-20,1.5488,0.61650,0.9220
2000-07-21,1.5511,0.61850,0.9363
2000-07-24,1.5555,0.61510,0.9332
2000-07-25,1.5538,0.62010,0.9410
...,...,...,...
2006-12-21,1.6032,0.67160,1.3178
2006-12-22,1.6029,0.67150,1.3192
2006-12-27,1.6058,0.67085,1.3159
2006-12-28,1.6058,0.67115,1.3173


### Tests

In [8]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [9]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [10]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 14.724s

OK
